# Домашнее задание 2: _Парсинг Авито_

In [ ]:
# для работы с вебом
from bs4 import BeautifulSoup
import requests
import random
import time

# для чистки текста
import re
import html

# для работы с файловой системой
import os

# для составления таблицы с метаинформацией
from datetime import datetime, timedelta
import pandas as pd

## Скачать страницу и обработать её содержимое

In [ ]:
yesterday = datetime.now() - timedelta(days=1)
yesterday.strftime("%m")

In [ ]:
def extract_meta(link, soup):
    page_meta = {
        "item": "",
        "id": "",
        "date": "",
        "seller": "",
        "address": ""
    }
    reg_date = re.compile("размещено (.*)?  ")
    reg_id = re.compile("№ ([0-9]+)")
    
    ad_item_str = link[46:]
    ad_metadata = soup.select("div.title-info-metadata-item")[0].text
    ad_item_id = re.search(reg_id, ad_metadata).group(1)
    
    ad_date = re.search(reg_date, ad_metadata).group(1)
    if "вчера" in ad_date:
        yesterday = datetime.now() - timedelta(days=1)
        ad_date = re.sub("вчера", yesterday.strftime("%d").lstrip("0") + " октября", ad_date)
    elif "сегодня" in ad_date:
        today = datetime.now()
        ad_date = re.sub("сегодня", today.strftime("%d").lstrip("0") + " октября", ad_date)
    ad_seller = soup.find("div", attrs={"class": "seller-info-name"}).find("a").text
    ad_address = soup.find("span", attrs={"itemtype": "http://schema.org/PostalAddress"}).text
    
    page_meta["item"] = ad_item_str
    page_meta["id"] = ad_item_id
    page_meta["date"] = ad_date
    page_meta["seller"] = clean_text(ad_seller)
    page_meta["address"] = clean_text(ad_address)
    
    return page_meta

In [ ]:
def parse_page(link):
    link_html = requests.get(link).text
    soup = BeautifulSoup(link_html, "lxml")
    ad_title = soup.select("h1.title-info-title")[0].text.strip()
    
    ad_description = soup.find("div", attrs={"class": "item-description"}).find("div")
    ad_paragraphs = list(ad_description.findChildren("p", recursive=False))
    ad_text = []
    ad_text = "\n".join([par.text for par in ad_paragraphs])
    
    total_text = ad_title + ad_text
    page_meta = extract_meta(link, soup)
    return total_text, page_meta

In [ ]:
def clean_text(raw_text):
    reg_tag = re.compile("<.*?>")
    reg_space = re.compile("\s{2,}")
    new_text = html.unescape(raw_text)
    new_text = reg_tag.sub("", new_text)
    new_text = reg_space.sub(" ", new_text)
    new_text = new_text.replace("\xa0", " ")
    return new_text

## Сохранить

In [ ]:
corpus = "." + os.sep + "Avito_Beauty_Corpus"
if not os.path.exists(corpus):
    os.mkdir(corpus)

In [ ]:
def save_page(link, contents):
    item = link[46:]
    page_path = corpus + os.sep + item +  ".txt"
    with open(page_path, "w", encoding="utf-8") as page_file:
        page_file.write(contents)

## Процесс

Сначала отдельно соберём ссылки на страницы с товарами и сохраним их в отдельный файл:


```python
start_link = "https://www.avito.ru/moskva/krasota_i_zdorove"
ready_links = []

for i in range(1, 200):
    # генерируем правильный адрес страницы, откуда будем таскать
    # ссылки на объявления
    if i > 1:
        current_link = start_link + "?p={}".format(i)
    else:
        current_link = start_link
    time.sleep(random.randint(1,20))
    print("Parsing page: {}".format(current_link))
    # забираем ссылки на объявления со страницы
    page = requests.get(current_link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        item_hrefs = [a["href"] for a in soup.select("a.item-description-title-link")]
        print("Found {} items".format(len(item_hrefs)))
        # собираем отдельно
        for item_href in item_hrefs:
            ready_link = "https://www.avito.ru" + item_href
            ready_links.append(ready_link)
    print("Done!")
```

Я уже это сделала, но этот код можно снова перенести в ячейку, и он будет работать. А теперь уже будем обкачивать сами страницы с объявлениями:

In [ ]:
with open("./links.txt", "r", encoding="utf-8") as f:
    links = [link.strip() for link in f.readlines()]
print("Начинаем собирать корпус из {} ссылок".format(len(links)))

In [ ]:
counter = 0
meta_list = {
    "item": [],
    "id": [],
    "date": [],
    "seller": [],
    "address": []
}

for item_link in links:
    sleep_time = random.randint(1, 20)
    print("Sleeping for {} sec.".format(sleep_time))
    time.sleep(sleep_time)
    print("Parsing webpage: {}".format(item_link))
    try:
        raw_text, meta = parse_page(item_link)
        new_text = clean_text(raw_text)
        save_page(item_link, new_text)
        for col in meta:
            meta_list[col].append(meta[col])
        counter += 1
        print("SUCCESS! Parsed {} pages".format(counter))
        break
    except:
        print("FAIL")

In [ ]:
items_df = pd.DataFrame(meta_list)
items_df.set_index("id", inplace=True)
items_df

In [ ]:
items_df.to_csv("./items_meta.csv", sep=";")